In [26]:
!pip install requests pandas python-dateutil tqdm lxml
!pip install nbainjuries  # from the mxufc29 project
!pip install html5lib


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [27]:
import requests
import json
import time
import random

NBA_LID, NBA_SPID = 5, 5

SBR_ENDPOINTS = [
    "https://www.oddstrader.com/odds-v2/odds-v2-service",
    "https://ms.production-us-east-1.bookmakersreview.com/ms-odds-v2/odds-v2-service",
]

def sbr_gql(query: str, variables: dict | None = None, endpoint: str | None = None, timeout=30, debug=False):
    payload = {
        "query": query,
        "variables": variables or {},
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/123.0.0.0 Safari/537.36"
        ),
        "Origin": "https://www.sportsbookreview.com",
        "Referer": "https://www.sportsbookreview.com/",
        "Accept-Language": "en-US,en;q=0.9",
    }

    endpoints = [endpoint] if endpoint else SBR_ENDPOINTS
    last_err = None

    for url in endpoints:
        try:
            r = requests.post(url, json=payload, headers=headers, timeout=timeout)
            if r.status_code != 200:
                raise RuntimeError(f"HTTP {r.status_code} from {url}\nResponse: {(r.text or '')[:800]}")
            out = r.json()
            if "errors" in out:
                if debug:
                    print("DEBUG payload sent:\n", json.dumps(payload, indent=2)[:2000])
                raise RuntimeError(out["errors"])
            return out["data"]
        except Exception as e:
            last_err = e
            time.sleep(0.25 + random.random() * 0.25)

    raise last_err



In [28]:
import re
import requests
from lxml import html as lh

SBR_UA = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
}

def matchup_ids_on_date(date_yyyy_mm_dd: str) -> list[int]:
    url = f"https://www.sportsbookreview.com/betting-odds/nba-basketball/?date={date_yyyy_mm_dd}"
    r = requests.get(url, headers=SBR_UA, timeout=30)
    r.raise_for_status()

    doc = lh.fromstring(r.text)
    hrefs = doc.xpath("//a/@href")

    ids = set()
    # match both matchup + line-history links (both carry the same numeric id)
    for h in hrefs:
        m = re.search(r"/nba-basketball/(?:line-history|matchup)/(\d+)/", h)
        if m:
            ids.add(int(m.group(1)))

    return sorted(ids)


In [29]:
def fetch_line_history_html(matchup_id: int) -> str:
    url = f"https://www.sportsbookreview.com/betting-odds/nba-basketball/line-history/{matchup_id}/"
    r = requests.get(url, headers=SBR_UA, timeout=30)
    r.raise_for_status()
    return r.text

def fetch_matchup_html(matchup_id: int) -> str:
    url = f"https://www.sportsbookreview.com/scores/nba-basketball/matchup/{matchup_id}/"
    r = requests.get(url, headers=SBR_UA, timeout=30)
    r.raise_for_status()
    return r.text

import json, re

def extract_next_data(html: str) -> dict | None:
    # Correct pattern: id="__NEXT_DATA__"
    m = re.search(r'<script[^>]+id="__NEXT_DATA__"[^>]*>\s*(\{.*?\})\s*</script>', html, re.S)
    if not m:
        return None
    try:
        return json.loads(m.group(1))
    except json.JSONDecodeError:
        return None

from collections.abc import Iterable

def walk_json(x):
    if isinstance(x, dict):
        yield x
        for v in x.values():
            yield from walk_json(v)
    elif isinstance(x, list):
        for it in x:
            yield from walk_json(it)

def find_line_history_records(next_data: dict) -> list[dict]:
    records = []
    for obj in walk_json(next_data):
        if not isinstance(obj, dict):
            continue
        keys = {str(k).lower() for k in obj.keys()}

        has_time = any(k in keys for k in ("ts","time","timestamp","date","createdat","updatedat")) or any("time" in k or "date" in k for k in keys)
        has_spread = any(k in keys for k in ("spread","hcap","handicap","line","points")) or any("spread" in k or "hcap" in k for k in keys)
        has_book = any(k in keys for k in ("book","sportsbook","sb","sportsbookname")) or any("book" in k or "sportsbook" in k for k in keys)

        if has_time and has_spread and has_book:
            records.append(obj)

    return records



In [30]:
import json
import re
import requests
import pandas as pd
from datetime import datetime, timezone

SBR_UA = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
}

def fetch_line_history_html(eid: int) -> str:
    url = f"https://www.sportsbookreview.com/betting-odds/nba-basketball/line-history/{eid}/"
    r = requests.get(url, headers=SBR_UA, timeout=30)
    r.raise_for_status()
    return r.text

def extract_next_data(html: str) -> dict:
    m = re.search(r'<script[^>]+id="__NEXT_DATA__"[^>]*>\s*(\{.*?\})\s*</script>', html, re.S)
    if not m:
        raise RuntimeError("No __NEXT_DATA__ script found")
    return json.loads(m.group(1))

def parse_fanduel_line_history(matchup_id: int) -> dict:
    """
    Return the raw Next.js pageProps dict for inspection.
    We'll extract the actual history list from it.
    """
    html = fetch_line_history_html(matchup_id)
    nd = extract_next_data(html)
    props = nd.get("props", {}).get("pageProps", {})
    return props



In [31]:
mid = 250647
props = parse_fanduel_line_history(mid)
print("pageProps keys:", list(props.keys())[:100])

def find_keys(obj, path=""):
    out = []
    if isinstance(obj, dict):
        for k,v in obj.items():
            p = f"{path}.{k}" if path else k
            kl = str(k).lower()
            if any(x in kl for x in ["history","line","odds","book","sportsbook","fanduel"]):
                out.append(p)
            out.extend(find_keys(v, p))
    elif isinstance(obj, list):
        for i,v in enumerate(obj[:50]):  # cap
            p = f"{path}[{i}]"
            out.extend(find_keys(v, p))
    return out

hits = find_keys(props)
print("HITS (first 80):")
print("\n".join(hits[:80]))



pageProps keys: ['regionCode', 'countryCode', 'lineHistoryModel', 'league', 'breadcrumbListObject', 'navVM', 'region']
HITS (first 80):
lineHistoryModel
lineHistoryModel.lineHistory
lineHistoryModel.lineHistory.gameView.consensus.homeMoneyLinePickPercent
lineHistoryModel.lineHistory.gameView.consensus.awayMoneyLinePickPercent
lineHistoryModel.lineHistory.oddsViews
lineHistoryModel.lineHistory.oddsViews[0].sportsbook
lineHistoryModel.lineHistory.oddsViews[0].sportsbookId
lineHistoryModel.lineHistory.oddsViews[0].openingLine
lineHistoryModel.lineHistory.oddsViews[0].currentLine
lineHistoryModel.lineHistory.oddsViews[0].moneyLineHistory
lineHistoryModel.lineHistory.oddsViews[0].moneyLineHistory[0].oddsDate
lineHistoryModel.lineHistory.oddsViews[0].moneyLineHistory[0].homeOdds
lineHistoryModel.lineHistory.oddsViews[0].moneyLineHistory[0].awayOdds
lineHistoryModel.lineHistory.oddsViews[0].moneyLineHistory[1].oddsDate
lineHistoryModel.lineHistory.oddsViews[0].moneyLineHistory[1].homeOdds
lin

In [32]:
import pandas as pd
from datetime import datetime, timezone

def parse_tipoff_from_linehistory_model(props: dict) -> datetime | None:
    """
    Try common locations for scheduled start. We'll expand if needed.
    """
    lh = props.get("lineHistoryModel", {}).get("lineHistory", {})
    gv = lh.get("gameView", {}) or {}
    # Common fields you may find: startDate, gameDate, eventDate, etc.
    for k in ["gameDate", "startDate", "startTime", "eventDate", "date"]:
        v = gv.get(k)
        if v:
            dt = pd.to_datetime(v, utc=True, errors="coerce")
            if not pd.isna(dt):
                return dt.to_pydatetime()
    return None

def fanduel_spread_history_from_props(props: dict) -> pd.DataFrame:
    odds_views = (
        props.get("lineHistoryModel", {})
             .get("lineHistory", {})
             .get("oddsViews", [])
    )
    if not isinstance(odds_views, list) or not odds_views:
        return pd.DataFrame()

    # find FanDuel
    fd = None
    for ov in odds_views:
        sb = str(ov.get("sportsbook") or "").strip().lower()
        if sb == "fanduel":
            fd = ov
            break
    if fd is None:
        return pd.DataFrame()

    hist = fd.get("spreadHistory") or []
    if not isinstance(hist, list) or not hist:
        return pd.DataFrame()

    df = pd.DataFrame(hist)

    # normalize time + spreads
    # oddsDate appears to be an ISO string
    df["ts"] = pd.to_datetime(df.get("oddsDate"), utc=True, errors="coerce")
    # homeSpread should exist (if not, we’ll infer later)
    if "homeSpread" in df.columns:
        df["home_spread"] = pd.to_numeric(df["homeSpread"], errors="coerce")
    elif "spread" in df.columns:
        df["home_spread"] = pd.to_numeric(df["spread"], errors="coerce")
    else:
        # fallback: you can inspect df.columns if this happens
        df["home_spread"] = pd.NA

    df = df.dropna(subset=["ts", "home_spread"]).sort_values("ts").reset_index(drop=True)

    # opening line (feature) from sportsbook node if present
    opening = fd.get("openingLine")
    # openingLine may be a dict or direct number depending on SBR
    if isinstance(opening, dict):
        # common key names
        for k in ["homeSpread", "spread", "hcap"]:
            if k in opening:
                opening = opening[k]
                break
    try:
        opening = float(opening) if opening is not None else None
    except Exception:
        opening = None

    df.attrs["opening_home_spread"] = opening
    df.attrs["sportsbookId"] = fd.get("sportsbookId")
    return df


In [33]:
def compute_targets_from_fd_history(fd_df: pd.DataFrame, tipoff_utc: datetime) -> tuple[float|None, float|None, float|None, float|None]:
    if fd_df is None or fd_df.empty:
        return None, None, None, None

    opening = fd_df.attrs.get("opening_home_spread")
    if opening is None:
        # fall back: first pre-tip value as opening
        opening = float(fd_df.iloc[0]["home_spread"])

    pre = fd_df[fd_df["ts"] < tipoff_utc].copy()
    if pre.empty:
        return None, None, None, None

    pre = pre.sort_values("ts")
    closing = float(pre.iloc[-1]["home_spread"])

    closing_diff = closing - float(opening)

    best = float(pre["home_spread"].max())   # best for home = higher spread
    worst = float(pre["home_spread"].min())  # worst for home = lower spread

    max_move_for = best - float(opening)
    max_move_away = float(opening) - worst

    if max_move_for < 0: max_move_for = 0.0
    if max_move_away < 0: max_move_away = 0.0

    return float(opening), float(closing_diff), float(max_move_for), float(max_move_away)


In [34]:
def parse_matchup_pageprops(matchup_id: int) -> dict:
    html = fetch_matchup_html(matchup_id)
    nd = extract_next_data(html)
    return nd.get("props", {}).get("pageProps", {})

def tipoff_utc_from_matchup(matchup_id: int) -> datetime:
    props = parse_matchup_pageprops(matchup_id)

    # Search a few common fields
    candidates = []
    def walk(o):
        if isinstance(o, dict):
            for k,v in o.items():
                kl = str(k).lower()
                if any(x in kl for x in ["start", "game", "tip", "date", "time"]) and isinstance(v, (str,int,float)):
                    candidates.append((k, v))
                walk(v)
        elif isinstance(o, list):
            for it in o[:50]:
                walk(it)

    walk(props)

    # Prefer ISO-like strings first
    for k,v in candidates:
        if isinstance(v, str) and ("t" in v.lower() and ":" in v):
            dt = pd.to_datetime(v, utc=True, errors="coerce")
            if not pd.isna(dt):
                return dt.to_pydatetime()

    # Next: epoch ms/sec
    for k,v in candidates:
        if isinstance(v, (int,float)) and v > 1e12:
            return datetime.fromtimestamp(float(v)/1000.0, tz=timezone.utc)
        if isinstance(v, (int,float)) and v > 1e9:
            return datetime.fromtimestamp(float(v), tz=timezone.utc)

    raise RuntimeError(f"Could not find tipoff time in matchup pageProps for matchup_id={matchup_id}. Sample candidates={candidates[:20]}")


In [35]:
def _to_datetime_utc(x) -> datetime | None:
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    if isinstance(x, (int, float)) and x > 1e12:
        # ms epoch
        return datetime.fromtimestamp(float(x)/1000.0, tz=timezone.utc)
    if isinstance(x, (int, float)) and x > 1e9:
        # sec epoch
        return datetime.fromtimestamp(float(x), tz=timezone.utc)
    s = str(x).strip()
    if not s:
        return None
    # Let pandas parse; assume UTC if no tz info
    dt = pd.to_datetime(s, utc=True, errors="coerce")
    if pd.isna(dt):
        return None
    return dt.to_pydatetime()

def normalize_line_history_df(raw: pd.DataFrame) -> pd.DataFrame:
    df = raw.copy()

    # book column
    book_col = None
    for c in df.columns:
        cl = str(c).lower()
        if cl in ("book", "sportsbook", "sports book", "sb"):
            book_col = c; break
        if "sportsbook" in cl or "book" in cl:
            book_col = c; break
    if book_col is None:
        # sometimes nested in JSON as sportsbook.name
        # just keep going and let user see failure
        raise RuntimeError(f"Cannot find book column. Columns={list(df.columns)}")

    # time column
    time_col = None
    for c in df.columns:
        cl = str(c).lower()
        if cl in ("time", "date", "timestamp", "ts"):
            time_col = c; break
        if "time" in cl or "date" in cl:
            time_col = c; break
    if time_col is None:
        raise RuntimeError(f"Cannot find time column. Columns={list(df.columns)}")

    # spread column (home handicap)
    spread_col = None
    for c in df.columns:
        cl = str(c).lower()
        if cl in ("spread", "hcap", "handicap", "line"):
            spread_col = c; break
        if "spread" in cl or "hcap" in cl or "handicap" in cl:
            spread_col = c; break
    if spread_col is None:
        raise RuntimeError(f"Cannot find spread column. Columns={list(df.columns)}")

    out = pd.DataFrame({
        "book": df[book_col].astype(str),
        "ts": df[time_col].map(_to_datetime_utc),
        "home_spread": pd.to_numeric(df[spread_col], errors="coerce"),
    })
    out = out.dropna(subset=["book", "ts", "home_spread"]).sort_values("ts")
    return out


In [36]:
import re
from lxml import html

CBS_TOP100_URL = "https://www.cbssports.com/nba/news/nba-top-100-players-2025-26-season/"  # :contentReference[oaicite:8]{index=8}

def fetch_cbs_top100_names() -> set[str]:
    r = requests.get(CBS_TOP100_URL, headers={"User-Agent": "Mozilla/5.0"}, timeout=30)
    r.raise_for_status()
    tree = html.fromstring(r.text)
    text = " ".join(tree.xpath("//text()"))
    # CBS pages often include “No. 1: Nikola Jokic” patterns
    names = set()
    for m in re.finditer(r"No\.\s*\d+\s*:\s*([A-Z][a-zA-Z\.\-']+\s+[A-Z][a-zA-Z\.\-']+)", text):
        names.add(m.group(1).strip())
    # If this returns small due to layout changes, you can swap to a more specific XPath later.
    return names

NOTABLE_PLAYERS = fetch_cbs_top100_names()
print("Notable players (seeded):", len(NOTABLE_PLAYERS))


Notable players (seeded): 0


In [37]:
import pandas as pd

def matchup_pageprops(matchup_id: int) -> dict:
    html = fetch_matchup_html(matchup_id)
    nd = extract_next_data(html)
    return nd.get("props", {}).get("pageProps", {})


def _walk(obj):
    if isinstance(obj, dict):
        yield obj
        for v in obj.values():
            yield from _walk(v)
    elif isinstance(obj, list):
        for it in obj[:200]:  # cap recursion
            yield from _walk(it)

def _find_first_dict_with_keys(props: dict, required_keys: set[str]) -> dict | None:
    req = {k.lower() for k in required_keys}
    for d in _walk(props):
        if isinstance(d, dict):
            keys = {str(k).lower() for k in d.keys()}
            if req.issubset(keys):
                return d
    return None

def extract_home_away_teams(props: dict) -> tuple[str|None, str|None]:
    """
    Try common shapes:
    - props.game.homeTeam / awayTeam
    - props.matchup.home / away
    - participants with isHome flag
    """
    # Try a "game" dict
    game = props.get("game") or props.get("matchup") or props.get("event")
    if isinstance(game, dict):
        for hk, ak in [("homeTeam","awayTeam"), ("home","away")]:
            h = game.get(hk); a = game.get(ak)
            if isinstance(h, dict) and isinstance(a, dict):
                hn = h.get("name") or h.get("nam") or h.get("fullName")
                an = a.get("name") or a.get("nam") or a.get("fullName")
                if hn and an:
                    return str(hn), str(an)

    # Search for participants
    d = _find_first_dict_with_keys(props, {"participants"})
    if d and isinstance(d.get("participants"), list):
        home = None; away = None
        for p in d["participants"]:
            if not isinstance(p, dict): 
                continue
            ih = p.get("ih") or p.get("isHome") or p.get("home")
            nm = None
            src = p.get("source") or p.get("team") or p
            if isinstance(src, dict):
                nm = src.get("nam") or src.get("name") or src.get("fullName")
            if ih is True:
                home = nm
            elif ih is False:
                away = nm
        return (str(home) if home else None, str(away) if away else None)

    return None, None


In [38]:
def extract_team_stats_features(props: dict) -> dict:
    """
    Returns columns like:
      stat_pts_for_home, stat_pts_for_away, stat_pts_against_home, ...
    using whatever the matchup page provides.
    """
    # Find a list-like block that looks like team stats rows
    # Common keys: "teamStats", "teamStatistics", "statistics", "statRows"
    candidates = []
    for d in _walk(props):
        if isinstance(d, dict):
            for k in d.keys():
                kl = str(k).lower()
                if kl in ("teamstats", "teamstatistics", "statistics", "statrows", "teamstatrows"):
                    v = d.get(k)
                    if isinstance(v, list) and v and isinstance(v[0], dict):
                        candidates.append(v)

    if not candidates:
        return {}

    rows = max(candidates, key=len)  # pick biggest list
    feats = {}

    for r in rows:
        keys = {str(k).lower() for k in r.keys()}
        # label key
        label = r.get("label") or r.get("name") or r.get("stat") or r.get("title")
        if not label:
            continue
        lab = str(label).strip().lower()
        lab = lab.replace("%", "pct").replace("/", "_per_").replace(" ", "_").replace("-", "_")

        # home/away values keys vary
        hv = r.get("home") or r.get("homeValue") or r.get("homeStat") or r.get("h")
        av = r.get("away") or r.get("awayValue") or r.get("awayStat") or r.get("a")

        # some rows are like {"values": {"home":..., "away":...}}
        if hv is None and isinstance(r.get("values"), dict):
            hv = r["values"].get("home")
            av = r["values"].get("away")

        # numeric parse if possible
        def to_num(x):
            if x is None: 
                return None
            s = str(x).strip().replace("%","")
            try:
                return float(s)
            except Exception:
                return None

        feats[f"stat_{lab}_home"] = to_num(hv) if to_num(hv) is not None else hv
        feats[f"stat_{lab}_away"] = to_num(av) if to_num(av) is not None else av

    return feats


In [39]:
mid = 362550
props = matchup_pageprops(mid)
home, away = extract_home_away_teams(props)
print("teams:", home, "vs", away)

feats = extract_team_stats_features(props)
print("n feats:", len(feats))
print(list(feats.items())[:20])


teams: None vs None
n feats: 0
[]


In [40]:
# Example usage pattern — adjust to the package’s exact API from its README :contentReference[oaicite:10]{index=10}
from nbainjuries import injury as inj

NOTABLE_PLAYERS = {
    # absolute superstars / perennial top-100 (add more over time)
    "LeBron James","Stephen Curry","Kevin Durant","Giannis Antetokounmpo","Nikola Jokic",
    "Luka Doncic","Joel Embiid","Jayson Tatum","Jimmy Butler","Kawhi Leonard",
    "Damian Lillard","Anthony Davis","Paul George","Kyrie Irving","James Harden",
    "Devin Booker","Shai Gilgeous-Alexander","Ja Morant","Trae Young","Donovan Mitchell"
}

import unicodedata
import re

def norm_name(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    s = s.lower().strip()
    s = re.sub(r"[^a-z\s\-']", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s

NOTABLE_NORM = {norm_name(x) for x in NOTABLE_PLAYERS}

# inj = injury()

def normalize_name(x: str) -> str:
    return re.sub(r"\s+", " ", x.strip())

def notable_injury_count_for_game(day: str, home_team: str, away_team: str, injuries_df: pd.DataFrame) -> int:
    """
    injuries_df should have columns like: date, team, player, status, etc.
    """
    ht = (home_team or "").lower().strip()
    at = (away_team or "").lower().strip()

    df = injuries_df
    df = df[df["date"] == day]

    # team matching (you may need mapping like "L.A. Lakers" -> "Los Angeles Lakers")
    df = df[df["team"].str.lower().isin([ht, at])]

    df["player_norm"] = df["player"].map(norm_name)
    df = df[df["player_norm"].isin(NOTABLE_NORM)]

    return int(df.shape[0])


def notable_injury_count(game_date: str, team_name: str, notable_set: set[str]) -> int:
    """
    game_date: YYYY-MM-DD (local date is fine; injuries are date-based)
    team_name: whatever your injury library expects (often full name or abbreviation)
    """
    df = inj.get_reportdata(date=game_date)  # per README usage :contentReference[oaicite:11]{index=11}
    if df is None or len(df) == 0:
        return 0

    # filter team
    tdf = df[df["team"].str.lower() == team_name.lower()].copy() if "team" in df.columns else df.copy()

    # define “injured enough to matter”
    status_col = "status" if "status" in tdf.columns else None
    if status_col:
        tdf = tdf[tdf[status_col].str.lower().isin({"out", "doubtful", "questionable"})]

    cnt = 0
    if "player" in tdf.columns:
        for p in tdf["player"].astype(str):
            if normalize_name(p) in notable_set:
                cnt += 1
    return cnt


In [41]:
from datetime import timedelta

def build_rows(start_date: str, end_date: str):
    start = dtparser.parse(start_date).date()
    end = dtparser.parse(end_date).date()

    all_rows = []
    d = start
    while d <= end:
        day = d.isoformat()
        mids = matchup_ids_on_date(day)
        print(day, "matchups:", len(mids), "sample:", mids[0] if mids else None)

        for mid in mids:
            # TODO: parse tipoff time (we can do it next)
            # For now, you can still compute peak/valley and later filter pre-tip once we parse tip.
            props_lh = parse_fanduel_line_history(mid)     # returns pageProps dict
            fd_df = fanduel_spread_history_from_props(props_lh)

            tip = tipoff_utc_from_matchup(mid)

            opening, closing_diff, max_for, max_away = compute_targets_from_fd_history(fd_df, tip)

            props_match = matchup_pageprops(mid)
            home_team, away_team = extract_home_away_teams(props_match)
            team_feats = extract_team_stats_features(props_match)

            # num_notable_inj = notable_injury_count(...)  # using normalized names + team mapping

            # if opening is None:
            #     skip["targets"] += 1
            #     continue

            row = {
                "date": day,
                "matchup_id": mid,
                "tipoff_utc": tip.isoformat(),
                "opening_home_spread": opening,
                "closing_line_diff": closing_diff,
                "max_move_for": max_for,
                "max_move_away": max_away,
            }

            all_rows.append(row)

        d = d + timedelta(days=1)

    return pd.DataFrame(all_rows)



df = build_rows("2021-10-31", "2021-11-02")
df.head()


2021-10-31 matchups: 5 sample: 250647
2021-11-01 matchups: 9 sample: 250652
2021-11-02 matchups: 5 sample: 250661


,date,matchup_id,tipoff_utc,opening_home_spread,closing_line_diff,max_move_for,max_move_away
0,2021-10-31,250647,2021-10-31T19:30:00+00:00,-5.5,1.0,1.5,0.0
1,2021-10-31,250648,2021-10-31T23:00:00+00:00,1.0,1.0,1.5,2.0
2,2021-10-31,250649,2021-10-31T23:00:00+00:00,1.0,4.0,4.0,0.0
3,2021-10-31,250650,2021-10-31T23:30:00+00:00,-12.5,0.5,1.0,0.0
4,2021-10-31,250651,2021-11-01T02:30:00+00:00,-12.0,1.0,1.5,0.0


In [42]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

target_cols = ["closing_line_diff", "max_move_for", "max_move_away"]
y = df[target_cols]

feature_cols = [c for c in df.columns if c not in target_cols and c not in ["date", "eid", "tipoff_utc", "home_team", "away_team"]]
X = df[feature_cols]

numeric = feature_cols

pre = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), numeric),
    ],
    remainder="drop",
)

model = MultiOutputRegressor(Ridge(alpha=1.0))

pipe = Pipeline([("pre", pre), ("model", model)])

# time series split (by date order)
df_sorted = df.sort_values(["date", "tipoff_utc"]).reset_index(drop=True)
X_sorted = df_sorted[feature_cols]
y_sorted = df_sorted[target_cols]

tscv = TimeSeriesSplit(n_splits=5)
for fold, (tr, te) in enumerate(tscv.split(X_sorted), 1):
    pipe.fit(X_sorted.iloc[tr], y_sorted.iloc[tr])
    score = pipe.score(X_sorted.iloc[te], y_sorted.iloc[te])  # R^2 (multioutput averaged)
    print(f"fold={fold} R2={score:.4f}")


fold=1 R2=-7.2250
fold=2 R2=-0.2452
fold=3 R2=-26.6205
fold=4 R2=-0.2959
fold=5 R2=-0.2160
